In [22]:
import pandas as pd

df_row = pd.read_csv("../data/공공자전거.csv",encoding='cp949')
df_row

,대여일자,대여시간,대여소번호,대여소명,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리(M),이용시간(분)
0,2024-03-01,0,1180,1180. 마곡엠밸리 15단지(1502동) 건너편,정기권,NaN,~10대,1,36.13,0.28,1216.47,17
1,2024-03-01,0,678,678.장안힐스테이트(아) 앞,정기권,NaN,~10대,1,61.85,0.56,2403.06,15
2,2024-03-01,0,108,108. 서교동 사거리,정기권,NaN,20대,1,26.45,0.27,1151.48,7
3,2024-03-01,0,733,733. 신정이펜하우스314동,정기권,NaN,20대,2,45.51,0.41,1757.10,15
4,2024-03-01,0,739,739. 신월사거리,정기권,NaN,20대,1,16.04,0.19,810.00,4
...,...,...,...,...,...,...,...,...,...,...,...,...
2885552,2024-03-31,23,2102,2102. 봉림교 교통섬,정기권,M,기타,1,13.13,0.12,510.00,4
2885553,2024-03-31,23,1427,1427. 용마산역 1번출구,정기권,M,기타,1,22.91,0.19,826.57,6
2885554,2024-03-31,23,1630,1630. 수연빌딩 앞,정기권,M,기타,1,77.36,0.73,3151.04,61
2885555,2024-03-31,23,1946,1946. 구로역 광장,정기권,M,기타,1,15.30,0.14,594.47,3


In [23]:
df_sliced = df_row.iloc[:,[1, 2, 7]]
df_sliced

,대여시간,대여소번호,이용건수
0,0,1180,1
1,0,678,1
2,0,108,1
3,0,733,2
4,0,739,1
...,...,...,...
2885552,23,2102,1
2885553,23,1427,1
2885554,23,1630,1
2885555,23,1946,1


In [24]:

file_path = "../data/공공자전거 대여소 정보.xlsx"

# '대여소현황' 시트에서 데이터 읽기 (실제 헤더는 3번째 줄이므로 header=2)
df = pd.read_excel(file_path, sheet_name='대여소현황', header=2)

# 'Unnamed: 0' → 대여소 번호, '자치구' 열만 선택
df_clean = df[['Unnamed: 0', '자치구']].copy()

# 대여소 번호가 숫자인 행만 필터링 (헤더나 빈행 제거)
df_clean = df_clean[pd.to_numeric(df_clean['Unnamed: 0'], errors='coerce').notnull()]

# 열 이름 정리
df_clean.rename(columns={'Unnamed: 0': '대여소번호'}, inplace=True)
df_clean['대여소번호'] = df_clean['대여소번호'].astype(int)

# 인덱스 재정렬
df_clean.reset_index(drop=True, inplace=True)

# 결과 확인
df_clean

,대여소번호,자치구
0,301,종로구
1,302,종로구
2,303,종로구
3,305,종로구
4,307,종로구
...,...,...
2761,4954,강동구
2762,4956,강동구
2763,4957,강동구
2764,4958,강동구


In [25]:
merged_df = df_sliced.merge(df_clean, on='대여소번호', how='left')

merged_df

,대여시간,대여소번호,이용건수,자치구
0,0,1180,1,강서구
1,0,678,1,동대문구
2,0,108,1,마포구
3,0,733,2,양천구
4,0,739,1,양천구
...,...,...,...,...
2885552,23,2102,1,관악구
2885553,23,1427,1,중랑구
2885554,23,1630,1,노원구
2885555,23,1946,1,구로구


In [15]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2885557 entries, 0 to 2885556
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   대여시간    int64 
 1   대여소번호   int64 
 2   자치구     object
dtypes: int64(2), object(1)
memory usage: 66.0+ MB


In [32]:
len(merged_df['대여소번호'].unique())

2730

In [30]:
merged_df[merged_df['대여시간']==0]['이용건수'].sum()

np.int64(59004)

----------

In [52]:
pivot_table = merged_df.pivot_table(
    index='자치구',
    columns='대여시간',
    values='이용건수',
    aggfunc='sum',
    fill_value=0
)

# 2. 자치구별 대여소수 구하기
station_counts = merged_df.drop_duplicates(['자치구', '대여소번호']).groupby('자치구')['대여소번호'].count()

# 3. 대여소수 컬럼 추가
pivot_table.insert(0, '대여소수', station_counts)

# 4. 컬럼 이름 정리: 시간 컬럼은 '0시', '1시' ... 형식으로
pivot_table.columns = ['대여소수'] + [f'{i}시' for i in pivot_table.columns[1:]]
hour_columns = [col for col in pivot_table.columns if "시" in col]
pivot_table[hour_columns] = pivot_table[hour_columns].div(31).astype(int)

pivot_table

,대여소수,0시,1시,2시,3시,4시,5시,6시,7시,8시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
자치구,,,,,,,,,,,,,,,,,,,,,
강남구,166,62,54,37,29,17,25,55,133,215,...,173,201,236,305,345,192,141,129,117,66
강동구,112,88,58,40,28,22,57,103,234,296,...,223,247,295,316,365,261,213,204,187,112
강북구,59,40,30,21,18,13,15,25,49,73,...,71,76,93,95,110,78,65,57,50,39
강서구,187,169,109,74,54,54,118,355,779,1026,...,510,604,762,1015,1167,720,548,494,409,255
관악구,70,62,53,38,26,21,24,45,92,148,...,109,124,138,145,163,130,112,107,99,68
광진구,81,106,76,42,31,24,42,91,209,437,...,271,280,347,383,420,290,238,234,205,145
구로구,111,75,44,32,21,20,38,83,244,334,...,185,221,262,323,407,233,177,155,131,89
금천구,74,25,19,11,8,6,15,39,115,199,...,73,74,95,177,225,101,70,55,44,28
노원구,141,127,84,59,45,31,60,125,301,356,...,351,406,476,492,496,371,300,283,263,153


In [53]:
pivot_table['0시'].sum()

np.int64(1865)

In [54]:
df_reset = pivot_table.reset_index()
df_reset.head()

,자치구,대여소수,0시,1시,2시,3시,4시,5시,6시,7시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,강남구,166,62,54,37,29,17,25,55,133,...,173,201,236,305,345,192,141,129,117,66
1,강동구,112,88,58,40,28,22,57,103,234,...,223,247,295,316,365,261,213,204,187,112
2,강북구,59,40,30,21,18,13,15,25,49,...,71,76,93,95,110,78,65,57,50,39
3,강서구,187,169,109,74,54,54,118,355,779,...,510,604,762,1015,1167,720,548,494,409,255
4,관악구,70,62,53,38,26,21,24,45,92,...,109,124,138,145,163,130,112,107,99,68


----------

In [55]:
import mysql.connector

gu_list = ['강남구', '강동구', '강북구', '강서구', '관악구', 
           '광진구', '구로구', '금천구', '노원구', '도봉구', 
           '동대문구', '동작구', '마포구', '서대문구', '서초구', 
           '성동구', '성북구', '송파구', '양천구', '영등포구', 
           '용산구', '은평구', '종로구', '중구', '중랑구']

conn = mysql.connector.connect(
host = "database-1.cb4o0sikaygu.ap-northeast-2.rds.amazonaws.com",
port = 3306,
user = "root",
password = "root762462",
database = "edadb"
)

In [56]:
df_db = df_reset.copy()
for i in range(len(gu_list)):
    df_db.iloc[i,0] = i

df_db['자치구']=df_db['자치구'].astype(int)

In [57]:
sql = "insert into bicycle_population values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s, %s,%s,%s,%s,%s,%s)"
cursor = conn.cursor(buffered=True)

In [58]:
import numpy as np

def convert_types(row):
    new_row = []
    for item in row:
        if isinstance(item, (np.int64, np.integer)):
            new_row.append(int(item))
        elif isinstance(item, (np.float64, np.floating)):
            new_row.append(float(item))
        else:
            new_row.append(item)
    return tuple(new_row)

data = [convert_types(row) for row in df_db.values]

cursor.executemany(sql, data)
conn.commit()